In [173]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import openpyxl

# 센서 설치 위치정보 데이터 전처리

In [174]:
location_df = pd.read_excel('./data/서울시 도시데이터 센서(S-DoT) 환경정보 설치 위치정보.xlsx')
location_df

/Users/m/Desktop/SKN24/.venv/lib/python3.14/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: 'S-DoT(2024년)'!$1:$1.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


,No,모델 시리얼(*),주소,좌표 구분코드,위도,경도,비고
0,1,V02Q1940059,서울특별시 강남구 일원동 688,W84,37.489530,127.075346,NaN
1,2,V02Q1940293,서울특별시 강남구 개포동 1221-3,W84,37.476401,127.048489,NaN
2,3,V02Q1940060,서울특별시 강남구 일원동 735-1,W84,37.483397,127.086234,NaN
3,4,V02Q1940079,서울특별시 강남구 일원동 707,W84,37.494102,127.082209,NaN
4,5,V02Q1940129,서울특별시 강남구 대치동 16-1,W84,37.497656,127.074399,NaN
...,...,...,...,...,...,...,...
1154,1155,OC3CL200169,서울특별시 중랑구 신내동 256-24,W84,37.620270,127.106154,NaN
1155,1156,OC3CL200225,서울특별시 중랑구 묵동 189-11,W84,37.609520,127.072813,NaN
1156,1157,OC3DL2200011,서울특별시 중랑구 면목동 49-10,W84,37.586201,127.094858,NaN
1157,1158,OC3DL2200010,서울특별시 중랑구 상봉동 481-2,W84,37.604117,127.090051,NaN


In [175]:
# 컬럼 공백 제거
location_df.columns = location_df.columns.str.strip()

# 필요한 컬럼만 뽑아오기 (시리얼 뽑아와서 자치구 별로 나누는 작업 필요)
use_columns = ['모델 시리얼(*)', '주소']
location_df = location_df[use_columns]

# 컬럼명 변경
location_df.columns = ['model_serial', 'address']


In [176]:
location_df

,model_serial,address
0,V02Q1940059,서울특별시 강남구 일원동 688
1,V02Q1940293,서울특별시 강남구 개포동 1221-3
2,V02Q1940060,서울특별시 강남구 일원동 735-1
3,V02Q1940079,서울특별시 강남구 일원동 707
4,V02Q1940129,서울특별시 강남구 대치동 16-1
...,...,...
1154,OC3CL200169,서울특별시 중랑구 신내동 256-24
1155,OC3CL200225,서울특별시 중랑구 묵동 189-11
1156,OC3DL2200011,서울특별시 중랑구 면목동 49-10
1157,OC3DL2200010,서울특별시 중랑구 상봉동 481-2


# 한파

In [177]:
# 데이터 로드

# 파일 경로 설정
path = './data/한파/S-DoT_NATURE_*.csv'
files = glob.glob(path)
df_list = [pd.read_csv(f, encoding='cp949') for f in files]
cold_wave_df = pd.concat(df_list, ignore_index=True)


/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/2351571185.py:6: DtypeWarning: Columns (0: 기온(℃), 1: 자외선(UVI)) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f, encoding='cp949') for f in files]
/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/2351571185.py:6: DtypeWarning: Columns (0: 흑구 운도(℃)) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f, encoding='cp949') for f in files]
/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/2351571185.py:6: DtypeWarning: Columns (0: 기온(℃), 1: 흑구 운도(℃)) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f, encoding='cp949') for f in files]
/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/2351571185.py:6: DtypeWarning: Columns (0: 초미세먼지 보정(㎍/㎥), 1: 미세먼지 보정(㎍/㎥)) have mixed types. Specify dtype option on import or set low_memory=False.
  df

In [178]:
# 컬럼 공백 제거
cold_wave_df.columns = cold_wave_df.columns.str.strip()

# 서울시 데이터 필터링
cold_wave_df = cold_wave_df[cold_wave_df['기관 명'] == '서울시']

# 필요한 컬럼만 뽑아오기 (시리얼 뽑아와서 자치구 별로 나누는 작업 필요)
use_columns = ['기관 명', '시리얼', '등록일자', '기온(℃)']
cold_wave_df = cold_wave_df[use_columns]

# 컬럼명 변경
cold_wave_df.columns = ['agency', 'serial', 'datetime', 'temp']


In [179]:
# 데이터 타입 변경 및 결측치 제거 (제거로 진행)
cold_wave_df['datetime'] = pd.to_datetime(cold_wave_df['datetime'], errors='coerce')
cold_wave_df['temp'] = pd.to_numeric(cold_wave_df['temp'], errors='coerce')
cold_wave_df = cold_wave_df.dropna(subset=['datetime', 'temp'])

# 이상치 제거
cold_wave_df = cold_wave_df[(cold_wave_df['temp'] > -25) & (cold_wave_df['temp'] < 45)]

# 날짜 컬럼 생성 (날짜별로 바꿔서 분석)
cold_wave_df['date'] = cold_wave_df['datetime'].dt.date

# 시리얼 번호도 같이 불러와야 하기 때문에 인덱스로 추가
idx_cold = cold_wave_df.groupby('date')['temp'].idxmin()
daily_cold = cold_wave_df.loc[idx_cold, ['date', 'serial', 'temp']].reset_index(drop=True)

In [ ]:
# 기준1 -12℃ 이하 2일 지속
daily_cold['yesterday_temp'] = daily_cold['temp'].shift(1)
rule_1 = (daily_cold['temp'] <= -12) & (daily_cold['yesterday_temp'] <= -12)

# 기준2 전날 보다 10도 이상 or 3도이하 일때
rule_2 = (daily_cold['yesterday_temp'] - daily_cold['temp'] >= 10) & (daily_cold['temp'] <= 3)

# 한파 여부 칼럼 추가
daily_cold['is_cold_wave'] = rule_1 | rule_2

# 임시 컬럼 삭제
daily_cold = daily_cold[['date', 'serial', 'temp', 'is_cold_wave']]


daily_cold

,date,serial,temp,is_cold_wave
0,2020-11-09,V02Q1940274,0.0,False
1,2020-11-10,V02Q1940274,0.0,False
2,2020-11-11,V02Q1940274,0.0,False
3,2020-11-12,V02Q1940274,0.0,False
4,2020-11-13,V02Q1940274,0.0,False
...,...,...,...,...
388,2022-12-24,OC3CL200086,-24.2,True
389,2022-12-25,OC3CL200086,-24.1,True
390,2022-12-26,OC3CL200086,-24.8,True
391,2022-12-27,OC3CL200086,-24.7,True


# 미세먼지

In [181]:
# 데이터 로드 인코딩 euc-kr, cp949 둘중하나 사용 (좀더 오래된 한글 인코딩 방식이 euc-kr이라고 함)
dust_df = pd.read_csv('./data/미세먼지/서울시 대기질 자료 제공_2020-2021.csv', encoding='cp949')

# 컬럼 공백 제거
dust_df.columns = dust_df.columns.str.strip()

In [182]:

# 평균값만 사용
dust_df = dust_df[dust_df['구분'] == '평균']

# 데이터 타입 변환 (errors 해석이 안되는 값들은 null 추가)
dust_df['일시'] = pd.to_datetime(dust_df['일시'], errors='coerce')

# 결측치 제거
dust_df = dust_df.dropna(subset=['일시', '미세먼지(PM10)', '초미세먼지(PM25)'])

# 컬럼명 변경
dust_df.columns = ['datetime', 'type', 'pm10', 'pm25']

# 날짜 컬럼 생성
dust_df['date'] = dust_df['datetime'].dt.date

In [183]:
# 일별 평균 계산
daily_dust = dust_df.groupby('date')[['pm10', 'pm25']].mean().reset_index()
daily_dust


,date,pm10,pm25
0,2020-01-01,32.916667,22.458333
1,2020-01-02,58.375000,39.958333
2,2020-01-03,67.166667,45.583333
3,2020-01-04,55.916667,38.166667
4,2020-01-05,51.833333,35.791667
...,...,...,...
726,2021-12-27,30.625000,17.208333
727,2021-12-28,50.250000,35.833333
728,2021-12-29,62.166667,45.000000
729,2021-12-30,28.791667,14.625000


# 폭염

In [184]:
# 데이터 로드

# 파일 경로 설정
path_heat = './data/폭염/S-DoT_NATURE_*.csv'
files = glob.glob(path_heat)
df_list = [pd.read_csv(f, encoding='cp949') for f in files]
heat_wave_df = pd.concat(df_list, ignore_index=True)


/var/folders/zh/lv2frb2s6_qbjc1lcfgty_th0000gn/T/ipykernel_21211/383167761.py:6: DtypeWarning: Columns (0: 구분, 1: 기온(℃), 2: 상대습도( %), 3: 풍향(°), 4: 풍속(m/s), 5: 돌풍 풍향(°), 6: 돌풍 풍속(m/s), 7: 조도(lux), 8: 자외선(UVI), 9: 소음(dB), 10: 진동_x(g), 11: 진동_y(g), 12: 진동_z(g), 13: 진동_x 최대(g), 14: 진동_y 최대(g), 15: 진동_z 최대(g), 16: 흑구 운도(℃), 17: 전송시간) have mixed types. Specify dtype option on import or set low_memory=False.
  df_list = [pd.read_csv(f, encoding='cp949') for f in files]


In [185]:
# 컬럼 공백 제거
heat_wave_df.columns = heat_wave_df.columns.str.strip()

# 서울시 데이터만 필터링
heat_wave_df = heat_wave_df[heat_wave_df['기관 명'] == '서울시']

# 필요한 컬럼만 추출
use_cols = ['기관 명', '시리얼', '등록일자', '기온(℃)']
heat_wave_df = heat_wave_df[use_cols]

# 맨뒤에 null 값만 들어간 기온 컬럼이 하나더 생성돼서 그 앞까지만 가져오기
heat_wave_df = heat_wave_df.iloc[:, [0, 1, 2, 3]]

# 컬럼명 변경
heat_wave_df.columns = ['agency', 'serial', 'datetime', 'temp']

In [186]:
# 데이터 타입 변경 및 결측치 제거
heat_wave_df['datetime'] = pd.to_datetime(heat_wave_df['datetime'], errors='coerce')
heat_wave_df['temp'] = pd.to_numeric(heat_wave_df['temp'], errors='coerce')
heat_wave_df = heat_wave_df.dropna(subset=['datetime', 'temp'])

# 이상치 제거 (공식 최고 기온: 39.6°C (2018년 8월 1일) 공식 최저 기온: -23.1°C (1927년 12월 31일) )
heat_wave_df = heat_wave_df[(heat_wave_df['temp'] > -25) & (heat_wave_df['temp'] < 45)]

# 날짜 컬럼 생성
heat_wave_df['date'] = heat_wave_df['datetime'].dt.date

# 시리얼 컬럼 값도 같이 찾아야 해서 인덱스로 추가
idx_heat = heat_wave_df.groupby('date')['temp'].idxmax()
daily_heat = heat_wave_df.loc[idx_heat, ['date', 'serial', 'temp']].reset_index(drop=True)

In [187]:
# 폭염 구별 칼럼 추가
daily_heat['heat_wave'] = daily_heat['temp'] >= 33

daily_heat.columns = ['date', 'heat_serial', 'max_temp', 'heat_wave']

# 데이터 결합

In [188]:
# 한파,미세먼지 데이터 병합 
cold_dust_df= pd.merge(
    daily_cold,
    daily_dust
)
cold_dust_df
final_df= pd.merge(
    daily_cold,
    daily_dust
)
final_df


,date,serial,temp,is_cold_wave,pm10,pm25
0,2020-11-09,V02Q1940274,0.0,False,25.833333,8.333333
1,2020-11-10,V02Q1940274,0.0,False,26.416667,11.125000
2,2020-11-11,V02Q1940274,0.0,False,39.833333,20.125000
3,2020-11-12,V02Q1940274,0.0,False,61.291667,38.500000
4,2020-11-13,V02Q1940274,0.0,False,72.958333,48.875000
...,...,...,...,...,...,...
208,2021-12-27,OC3CL200187,-23.4,True,30.625000,17.208333
209,2021-12-28,OC3CL200187,-22.3,True,50.250000,35.833333
210,2021-12-29,OC3CL200187,-16.5,True,62.166667,45.000000
211,2021-12-30,OC3CL200187,-20.8,True,28.791667,14.625000
